In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Análise de Sobrevivência - Rotatividade de Funcionários

Este é um estudo baseado em análise de sobrevivência, com o objetivo de determinar o tempo até um funcionário deixar uma companhia. Os dados utilizados são fornecidos [neste blog](https://edwvb.blogspot.com/2017/10/employee-turnover-how-to-predict-individual-risks-of-quitting.html) e, a princípio, traram-se de dados reais (não localizei a origem precisa desses dados). 

### Informações disponíveis

O conjunto de dados é composto das seguintes informações:

- `stag`: Experiência do Funcionário. Corresponde ao tempo t de falha ou tempo de censura do estudo. 
- `event`: Indica se o evento ocorreu no tempo t. Caso seja 1, indica falha, ou seja, o funcionário em questão deixou a empresa no tempo t. 
- `gender`: gênero do funcionário.
- `age`: Idade do funcionário.
- `industry`: Industria de atuação do funcionário.
- `profession`: profissão do funcionário.
- `traffic`: De qual canal o funcionário ingressou para a empresa
- `coach`: Presença de um coach 
- `head_gender`: Gênero do supervisor ao qual o funcionario responde
- `greywage`: O salário não parece para as autoridades fiscais. Greywage na Rússia ou na Ucrânia significa que o empregador (empresa) paga apenas uma pequena quantia de salário acima do salário mínimo (white).
- `way`: Forma que o funcionário se desloca para o escritório.
- `extraversion`: Escala segundo o teste Big5.
- `independ`: Escala segundo o teste Big5.
- `selfcontrol`: Escala segundo o teste Big5.
- `anxiety`: Escala segundo o teste Big5.
- `novator`: Escala segundo o teste Big5.

---

## 1. Preparando o terreno

Nesta primeira etapa, vamos preparar tudo o que precisamos para a análise de sobrevivência tomando os seguintes passos:

- 1.0 Carregando biblitecas e dados e criação de um dicionário de dados;
- 1.1 Visão geral da base: identificação de valores faltantes, identificação de categorias distintas, distribuições e sumarização dos dados numéricos;
- 1.2 Dados categóricos 1: variáveis binárias;
- 1.3 Dados categóricos 2: variáveis com mais de duas categorias;


### 1.1 Carregando os dados e criando um dicionário de dados

In [ ]:
# Módulos utilizados
import pandas as pd
import matplotlib.pyplot as plt

# Base bruta
raw_df = pd.read_csv('/kaggle/input/employee-turnover/turnover.csv')

# Cópia da base bruta para transformações
df = raw_df.copy()

# Visualização das 5 primeiras linhas
df.head()

O dicionário de dados será utilizado para facilitar as manipulações no processo de analise. Para cada variável, conterá as informações:

- `hole`: meio que a variável se enquadra, podendo ser `target` para variáveis respostas, `bin` para variáveis categóricas que apresentam apenas dois valores distintos, `categorical` para variáveis categoricas com mais de dois valores distintos e `value` para valores inteiros ou números reais. 
- `type`: representa o tipo em que a variável se apresenta;
- `kind`: complementar do `hole`, indica se é variável independente (`variable`), tempo de falha (`failure_time`) ou categoria alvo (`target`).

In [ ]:
## Dicionário de Dados

data_dict = pd.DataFrame(
    {
        'stag': {'hole': 'target', 'type': 'interval', 'kind': 'failure_time'},
        'event': {'hole': 'target', 'type': 'bool', 'kind': 'target'},
        'gender': {'hole': 'bin', 'type': 'str', 'kind': 'variable'},
        'age': {'hole': 'value', 'type': 'integer', 'kind': 'variable'},
        'industry': {'hole': 'categorical', 'type': 'str', 'kind': 'variable'},
        'profession': {'hole': 'categorical', 'type': 'str', 'kind': 'variable'},
        'traffic': {'hole': 'categorical', 'type': 'str', 'kind': 'variable'},
        'coach': {'hole': 'bin', 'type': 'str', 'kind': 'variable'},
        'head_gender': {'hole': 'bin', 'type': 'str', 'kind': 'variable'},
        'greywage': {'hole': 'bin', 'type': 'str', 'kind': 'variable'},
        'way': {'hole': 'categorical', 'type': 'str', 'kind': 'variable'},
        'extraversion': {'hole': 'value', 'type': 'interval', 'kind': 'variable'},
        'independ': {'hole': 'value', 'type': 'interval', 'kind': 'variable'},
        'selfcontrol': {'hole': 'value', 'type': 'interval', 'kind': 'variable'},
        'anxiety': {'hole': 'value', 'type': 'interval', 'kind': 'variable'},
        'novator': {'hole': 'value', 'type': 'interval', 'kind': 'variable'},
    }
).T

data_dict

### 1.1  Visão geral da base

Identificação de valores faltantes, identificação de categorias distintas, distribuições e sumarização dos dados numéricos.

Presença de valores ausentes:

In [ ]:
df.isnull().sum() / df.shape[0]

Número de categorias distintas nas variáveis categóricas:

In [ ]:
df[data_dict[data_dict['hole'] == 'categorical'].index].nunique()

Distribuiução de variáveis numéricas:

In [ ]:
num_cols = data_dict[data_dict['hole'] == 'value'].index

for c in num_cols:
    df[c].plot.hist(bins=20)
    plt.title('Histogram of ' + c)
    plt.show()

Sumarização das variáveis numéricas:

In [ ]:
df[num_cols].describe()

Proporção da variável alvo:

In [ ]:
df[data_dict[data_dict['kind'] == 'target'].index].value_counts() / len(df)

### 1.2 Dados categóricos 1: variáveis binárias

Vamos modificar para que sejam representados pelos valores 0 e 1, ao invés de letras ou labels. 

- `gender` será modificada para `is_female`, onde 1 representará funcionário do sexo feminino e 0 do sexo masculino;
- `head_gender` será modificada para `is_female_head`, onde 1 representará funcionário que possui supervisor do sexo feminino e 0 do sexo masculino;
- `coach` será modificado de (yes, no) para (1, 0), adicionando a coluna `head_is_coach` que indicará se o coach foi o próprio supervisor;
- `greywage` será modificado para `is_greywage`, onde a categoria `white` receberá valor 0 e `gray` receberá valor 1. 

In [ ]:
df['is_female'] = [1 if gender == 'f' else 0 for gender in df['gender']]
df.drop('gender', axis=1, inplace=True)
data_dict = data_dict.append(pd.DataFrame({'is_female': {'hole': 'bin', 'type': 'bool', 'kind': 'variable'}}).T)
data_dict.drop('gender', axis=0, inplace=True)


In [ ]:
df['head_is_female'] = [1 if gender == 'f' else 0 for gender in df['head_gender']]
df.drop('head_gender', axis=1, inplace=True)
data_dict = data_dict.append(pd.DataFrame({'head_is_female': {'hole': 'bin', 'type': 'bool', 'kind': 'variable'}}).T)
data_dict.drop('head_gender', axis=0, inplace=True)


In [ ]:
df['head_is_coach'] = [1 if coach == 'my head' else 0 for coach in df['coach']]
df['coach'] = [1 if coach == 'yes' else 0 for coach in df['coach']]
data_dict = data_dict.append(pd.DataFrame({'head_is_coach': {'hole': 'bin', 'type': 'bool', 'kind': 'variable'}}).T)

In [ ]:
df['is_greywage'] = [1 if greywage == 'grey' else 0 for greywage in df['greywage']]
df.drop('greywage', axis=1, inplace=True)
data_dict = data_dict.append(pd.DataFrame({'is_greywage': {'hole': 'bin', 'type': 'bool', 'kind': 'variable'}}).T)
data_dict.drop('greywage', axis=0, inplace=True)


In [ ]:
df.head()

In [ ]:
data_dict